# MAXIMIZA LOS 1

Implementad un algoritmo genético que encuentre una solución que maximice el número de
dígitos con valor 1 en una cadena binaria (el tamaño de la cadena debe poder elegirse al
ejecutar el algoritmo).

In [4]:
import random

class GeneticAlgorithm(object):
    def __init__(self, genetics):
        self.genetics = genetics
        pass

    def run(self):
        population = self.genetics.initial()
        while True:
            fits_pops = [(self.genetics.fitness(ch),  ch) for ch in population]
            if self.genetics.check_stop(fits_pops): break
            population = self.next(fits_pops)
            pass
        return population

    def next(self, fits):
        parents_generator = self.genetics.parents(fits)
        size = len(fits)
        nexts = []
        while len(nexts) < size:
            parents = next(parents_generator)
            cross = random.random() < self.genetics.probability_crossover()
            children = self.genetics.crossover(parents) if cross else parents
            for ch in children:
                mutate = random.random() < self.genetics.probability_mutation()
                nexts.append(self.genetics.mutation(ch) if mutate else ch)
                pass
            pass
        return nexts[0:size]
    pass

class GeneticFunctions(object):
    def probability_crossover(self):
        r"""returns rate of occur crossover(0.0-1.0)"""
        return 1.0

    def probability_mutation(self):
        r"""returns rate of occur mutation(0.0-1.0)"""
        return 0.0

    def initial(self):
        r"""returns list of initial population
        """
        return []

    def fitness(self, chromosome):
        r"""returns domain fitness value of chromosome
        """
        return len(chromosome)

    def check_stop(self, fits_populations):
        r"""stop run if returns True
        - fits_populations: list of (fitness_value, chromosome)
        """
        return False

    def parents(self, fits_populations):
        r"""generator of selected parents
        """
        gen = iter(sorted(fits_populations))
        while True:
            f1, ch1 = next(gen)
            f2, ch2 = next(gen)
            yield (ch1, ch2)
            pass
        return

    def crossover(self, parents):
        r"""breed children
        """
        return parents

    def mutation(self, chromosome):
        r"""mutate chromosome
        """
        return chromosome
    pass

if __name__ == "__main__":
    """
    example: Mapped guess prepared Text
    """
    class Maximize_ones(GeneticFunctions):
        def __init__(self, target_text,
                     limit=2000 ##Limits the number of iterations
                     , size=400 ##Sets the population size
                     , prob_crossover=0.9 ##Probability of crossover
                     , prob_mutation=0.2): ##Probability of mutation
            self.target = self.text2chromo(target_text) ##target in fitness function
            self.counter = 0 ##Counter to start iterations

            self.limit = limit 
            self.size = size
            self.prob_crossover = prob_crossover
            self.prob_mutation = prob_mutation
            pass

        # GeneticFunctions interface impls
        def probability_crossover(self):
            return self.prob_crossover

        def probability_mutation(self):
            return self.prob_mutation

        def initial(self):
            return [self.random_chromo() for j in range(self.size)]

        def fitness(self, chromo):
            # larger is better, matched == 0
            return -sum(abs(c - t) for c, t in zip(chromo, self.target))

        def check_stop(self, fits_populations):
            self.counter += 1
            if self.counter % 10 == 0:
                best_match = list(sorted(fits_populations))[-1][1]
                fits = [f for f, ch in fits_populations]
                best = max(fits)
                worst = min(fits)
                ave = sum(fits) / len(fits)
                print(
                    "[G %3d] score=(%4d, %4d, %4d): %r" %
                    (self.counter, best, ave, worst,
                    best_match))
                pass
            return self.counter >= self.limit

        def parents(self, fits_populations):
            while True:
                father = self.tournament(fits_populations)
                mother = self.tournament(fits_populations)
                yield (father, mother)
                pass
            pass

        def crossover(self, parents):
            father, mother = parents
            index1 = random.randint(1, len(self.target) - 2)
            index2 = random.randint(1, len(self.target) - 2)
            if index1 > index2: index1, index2 = index2, index1
            child1 = father[:index1] + mother[index1:index2] + father[index2:]
            child2 = mother[:index1] + father[index1:index2] + mother[index2:]
            return (child1, child2)

        def mutation(self, chromosome):
            index = random.randint(0, len(self.target) - 1)
            vary = random.randint(-10, 10)
            mutated = list(chromosome)
            mutated[index] += vary
            return mutated

        # internals
        def tournament(self, fits_populations):
            alicef, alice = self.select_random(fits_populations)
            bobf, bob = self.select_random(fits_populations)
            return alice if alicef > bobf else bob

        def select_random(self, fits_populations):
            return fits_populations[random.randint(0, len(fits_populations)-1)]

        def text2chromo(self, text):
            return [ord(ch) for ch in text]
        def chromo2text(self, chromo):
            return "".join(chr(max(1, min(ch, 255))) for ch in chromo)

        def random_chromo(self):
            return [random.randint(1, 255) for i in range(len(self.target))]
        pass

    GeneticAlgorithm(Maximize_ones("No me estoy enterando de nada de nada pero al menos lo intentamo")).run()
    pass

[G  10] score=(-2427, -3301, -3895): [136, 166, 82, 123, 84, 6, 135, 83, 10, 157, 52, 107, 117, 70, 153, 89, 74, 114, 128, 126, 100, 42, 200, 38, 65, 94, 63, 64, 97, 72, 108, 111, 243, 183, 78, 89, 86, 17, 132, 173, 58, 231, 77, 99, 122, 117, 143, 79, 76, 161, 123, 11, 171, 132, 94, 102, 78, 152, 130, 129, 84, 66, 106, 87]
[G  20] score=(-1856, -2208, -2735): [82, 152, 105, 106, 84, 6, 135, 91, 47, 144, 112, 23, 106, 34, 113, 89, 74, 114, 128, 126, 96, 42, 94, 186, 41, 118, 103, 74, 65, 11, 100, 166, 7, 78, 78, 89, 86, 17, 132, 173, 58, 106, 102, 99, 122, 117, 143, 79, 76, 161, 123, 11, 171, 132, 94, 102, 133, 152, 130, 129, 84, 167, 67, 139]
[G  30] score=(-1312, -1534, -1821): [113, 109, 16, 106, 84, 54, 92, 91, 91, 103, 162, 31, 117, 102, 111, 89, 74, 104, 106, 112, 140, 42, 85, 152, 22, 111, 103, 74, 97, 72, 108, 166, 7, 78, 86, 91, 120, 17, 132, 119, 102, 80, 33, 99, 94, 55, 113, 98, 134, 191, 178, 11, 171, 132, 18, 102, 133, 152, 130, 87, 95, 133, 151, 109]
[G  40] score=(-984, -

[G 270] score=( -18,  -22,  -32): [78, 109, 32, 109, 100, 31, 101, 115, 116, 111, 121, 31, 101, 110, 116, 101, 115, 97, 110, 99, 111, 32, 101, 101, 32, 110, 97, 100, 97, 32, 100, 101, 31, 110, 97, 100, 97, 34, 112, 101, 114, 110, 32, 97, 108, 32, 109, 100, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 108, 116, 98, 109, 109]
[G 280] score=( -13,  -18,  -31): [78, 109, 32, 109, 100, 31, 101, 115, 116, 111, 121, 31, 101, 110, 116, 101, 115, 97, 110, 99, 111, 32, 101, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 110, 32, 97, 108, 32, 109, 100, 110, 111, 115, 31, 108, 111, 32, 105, 110, 116, 101, 110, 116, 98, 109, 110]
[G 290] score=( -10,  -13,  -26): [78, 109, 32, 109, 102, 31, 101, 115, 116, 111, 121, 31, 101, 110, 116, 101, 115, 97, 110, 99, 111, 32, 101, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 98, 109, 110

[G 530] score=(   0,   -1,  -10): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 540] score=(   0,   -1,  -12): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 550] score=(   0,   -1,  -14): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 

[G 810] score=(   0,   -1,  -10): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 820] score=(   0,   -1,  -13): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 830] score=(   0,   -1,  -17): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 

[G 1070] score=(   0,   -1,  -14): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1080] score=(   0,   -1,  -10): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1090] score=(   0,   -1,  -15): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 10

[G 1350] score=(   0,    0,  -13): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1360] score=(   0,   -1,  -15): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1370] score=(   0,   -1,  -13): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 10

[G 1640] score=(   0,   -1,  -15): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1650] score=(   0,   -1,  -14): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1660] score=(   0,   -1,  -10): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 10

[G 1920] score=(   0,   -1,  -19): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1930] score=(   0,   -1,  -10): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 109, 111]
[G 1940] score=(   0,   -1,  -12): [78, 111, 32, 109, 101, 32, 101, 115, 116, 111, 121, 32, 101, 110, 116, 101, 114, 97, 110, 100, 111, 32, 100, 101, 32, 110, 97, 100, 97, 32, 100, 101, 32, 110, 97, 100, 97, 32, 112, 101, 114, 111, 32, 97, 108, 32, 109, 101, 110, 111, 115, 32, 108, 111, 32, 105, 110, 116, 101, 110, 116, 97, 10

In [13]:
# File: oneMaxTests.py
#    from chapter 2 of _Genetic Algorithms with Python_
#
# Author: Clinton Sheppard <fluentcoder@gmail.com>
# Copyright (c) 2016 Clinton Sheppard
#
# Licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.  See the License for the specific language governing
# permissions and limitations under the License.

import datetime
import unittest

import random
import statistics
import sys
import time

def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)


def _mutate(parent, geneSet, get_fitness):
    childGenes = parent.Genes[:]
    index = random.randrange(0, len(parent.Genes))
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness)


def get_best(get_fitness, targetLen, optimalFitness, geneSet, display):
    random.seed()
    bestParent = _generate_parent(targetLen, geneSet, get_fitness)
    display(bestParent)
    if bestParent.Fitness >= optimalFitness:
        return bestParent
    while True:
        child = _mutate(bestParent, geneSet, get_fitness)
        if bestParent.Fitness >= child.Fitness:
            continue
        display(child)
        if child.Fitness >= optimalFitness:
            return child
        bestParent = child


class Chromosome:
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness


class Benchmark:
    @staticmethod
    def run(function):
        timings = []
        stdout = sys.stdout
        for i in range(100):
            sys.stdout = None
            startTime = time.time()
            function()
            seconds = time.time() - startTime
            sys.stdout = stdout
            timings.append(seconds)
            mean = statistics.mean(timings)
            if i < 10 or i % 10 == 9:
                print("{} {:3.2f} {:3.2f}".format(
                    1 + i, mean,
                    statistics.stdev(timings, mean) if i > 1 else 0))

def get_fitness(genes):
    return genes.count(1)


def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}...{}\t{:3.2f}\t{}".format(
        ''.join(map(str, candidate.Genes[:15])),
        ''.join(map(str, candidate.Genes[-15:])),
        candidate.Fitness,
        timeDiff))


class OneMaxTests(unittest.TestCase):
    def test(self, length=100):
        geneset = [0, 1]
        startTime = datetime.datetime.now()

        def fnDisplay(candidate):
            display(candidate, startTime)

        def fnGetFitness(genes):
            return get_fitness(genes)

        optimalFitness = length
        best = get_best(fnGetFitness, length, optimalFitness,
                                geneset, fnDisplay)
        self.assertEqual(best.Fitness, optimalFitness)

    def test_benchmark(self):
        Benchmark.run(lambda: self.test(4000))


if __name__ == '__main__':
    unittest.main()

E
ERROR: C:\Users\a (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\a'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

C:\Users\a.berian\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
